<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/Gradient_Decent_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pylab as plt

# Gradient, softmax, corss_entorpy_error 구현

In [ ]:
def _numerical_gradient_no_batch(f, x): # 수치 gradient, 앞의 수치미분과 같은 개념
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성(이 경우 x와 같은 차원의 제로 벡터)

    # gradient는 각 축에 대한 방향 미분을 모아놓은 벡터이므로 각 방향에 대한 수치미분을 grad 벡터에 차례로 대입하는 과정이다.
    for idx in range(x.size):
        tmp_val = x[idx]

        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] =  (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원

    return grad

In [ ]:
def numerical_gradient(f, X): # 배치처리를 한 경우(행렬, 텐서 미분)
    if X.ndim == 1: # 1차원, 그러니까 행렬로 묶이지 않은 벡터인 경우
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)

        for idx, x in enumerate(X): # enumerate 설명1, 행벡터를 차례로 불러와 미분한다.
            grad[idx] = _numerical_gradient_no_batch(f, x)

        return grad

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

In [ ]:
def softmax(x):
    if x.dim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

# Gradient Decent 구현

$ x_{n+1} = x_{n} - \eta \nabla f(x_{n}) $

$\eta = $ learning rate(학습률)

In [ ]:
def gradient_decent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append( x.copy() ) # 연동카피를 방지하기 위해 사용한다.

        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)

# Gradient Decent 테스트

In [ ]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [ ]:
init_x = np.array([-3.0, 4.0]) # 초기 좌표
lr = 0.1
step_num = 20
x, x_history = gradient_decent(function_2, init_x, lr = lr, step_num = step_num)

x = np.arange(-4.0, 5.0, 0.25)
y = np.arange(-4.0, 5.0, 0.25)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

plt.plot( [-5, 5], [0, 0], '--b') # x축
plt.plot( [0, 0], [-5, 5], '--b') # y축
plt.plot(x_history[:,0], x_history[:,1], 'o')
plt.plot
plt.contour(X, Y, Z)

plt.xlim(-3.5, 3.5)
plt.ylim(-4.5, 4.5)
plt.xlabel("X0")
plt.ylabel("Y0")
plt.show()

$\eta$ (학습률) 이 너무 작은경우 극소점까지 도달하지 못 할 수 있다. gradient decent에서 보폭은 $ \eta \Vert \nabla f(x) \Vert $ 인데 gradient decent가 진행될 수록 $ \Vert \nabla f(x) \Vert $도 같이 작아지므로 극소점까지 도달하기까지 굉장히 많은 step 이 필요하게 된다.

In [ ]:
init_x = np.array([-3.0, 4.0]) # 초기 좌표
lr = 0.01
step_num = 20
x, x_history = gradient_decent(function_2, init_x, lr = lr, step_num = step_num)

plt.plot( [-5, 5], [0, 0], '--b') # x축
plt.plot( [0, 0], [-5, 5], '--b') # y축
plt.plot(x_history[:,0], x_history[:,1], 'o')
plt.plot

plt.xlim(-3.5, 3.5)
plt.ylim(-4.5, 4.5)
plt.xlabel("X0")
plt.ylabel("Y0")
plt.show()

$\eta$(학습률)이 너무 큰경우 극소로 향하지 않고 오히려 발산할 수 있다.

In [ ]:
init_x = np.array([-3.0, 4.0]) # 초기 좌표
lr = 1.01
step_num = 20
x, x_history = gradient_decent(function_2, init_x, lr = lr, step_num = step_num)

plt.plot( [-5, 5], [0, 0], '--b') # x축
plt.plot( [0, 0], [-5, 5], '--b') # y축
plt.plot(x_history[:,0], x_history[:,1], 'o')
plt.plot

plt.xlim(-3.5, 3.5)
plt.ylim(-4.5, 4.5)
plt.xlabel("X0")
plt.ylabel("Y0")
plt.show()

# 간단한 신경망에 적용해보기

simple net 입력층 2개 출력층 3개로 이루어진 간다한 신경망으로 bias(편향)은 제외한다.

In [ ]:
class simpleNet:
    def __init__(self): # self는 인스턴스 생성시 인스턴스의 이름이다.
        self.W = np.random.randn(2,3) # (randn(random normal))정규분포로 초기화. 정규분포의 확률밀도함수에따라 랜덤한 숫자를 뽑아 2*3행렬을 반환한다.
                                      # randn은 표준정규분포로, rand는 균등분포로 초기화 할 수 있다.

    def predict(self, x):             # x는 imput data로 2차원 벡터일 것 이다.
        return np.dot(x, self.W)      # 2차원 벡터와 2*3행렬을 내적하므로 3차원 벡터가 반환된다.

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z) # 추론값을 확률벡터로 반환
        loss = cross_entropy_error(y, t) # t는 label이다.

        return loss

In [ ]:
net = simpleNet()

In [ ]:
print(net.W)

In [ ]:
x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)